In [1]:
pip install langchain_groq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
       temperature=0,
       groq_api_key="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",
       model_name = "llama-3.3-70b-versatile" # Replace with the supported model ID
   )

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://builtin.com/job/software-engineer-ii-aiml-platform/3674238")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.









SOFTWARE ENGINEER II, AIML Platform - Nike, Inc. | Built In

























View All Jobs

                            For Employers
                        
                                Join
                            


                                Log In
                            
JobsCompaniesRemoteArticlesBest Places To Work
                                    My items
                                

















Nike, Inc.



SOFTWARE ENGINEER II, AIML Platform



Sorry, this job was removed at 04:56 a.m. (CST) on Thursday, Jan 09, 2025













United States of America







Other • Retail
At NIKE, Inc., technology is laying the foundation for our digital transformation and direct-to-consumer strategy. 








The Role

WHO WE ARE LOOKING FOR
We are looking for a Software Engineer to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the inte

In [5]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is form the carrer's page of a website.
    Your job is to extract the job posting and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills' and 'description'.
    only return the JSON.
    """

)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
    "role": "SOFTWARE ENGINEER II, AIML Platform",
    "experience": "2+ years professional software development experience",
    "skills": [
        "Computer Science fundamentals",
        "Programming experience with at least one modern language such as Java, Python, Golang",
        "Experience with Infrastructure as Code / DevOps practices and technologies like Terraform",
        "Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)",
        "Experience developing and using microservices"
    ],
    "description": "We are looking for a Software Engineer to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning. You will also work collaboratively with the data science and analytics teams to ship scalable, performant and reliable solutions."
}
```


In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'SOFTWARE ENGINEER II, AIML Platform',
 'experience': '2+ years professional software development experience',
 'skills': ['Computer Science fundamentals',
  'Programming experience with at least one modern language such as Java, Python, Golang',
  'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
  'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
  'Experience developing and using microservices'],
 'description': 'We are looking for a Software Engineer to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning. You will also work collaboratively with the data science and analytics teams to ship scalable, performant and reliable solutions.'}

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
pip install chromadb

In [11]:

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=["Experience in python", "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [13]:
job = json_res  # Assign json_res to job
job['skills']

['Computer Science fundamentals',
 'Programming experience with at least one modern language such as Java, Python, Golang',
 'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
 'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
 'Experience developing and using microservices']

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [15]:
job

{'role': 'SOFTWARE ENGINEER II, AIML Platform',
 'experience': '2+ years professional software development experience',
 'skills': ['Computer Science fundamentals',
  'Programming experience with at least one modern language such as Java, Python, Golang',
  'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
  'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
  'Experience developing and using microservices'],
 'description': 'We are looking for a Software Engineer to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning. You will also work collaboratively with the data science and analytics teams to ship scalable, performant and reliable solutions.'}

In [16]:
job = json_res
job['skills']

['Computer Science fundamentals',
 'Programming experience with at least one modern language such as Java, Python, Golang',
 'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
 'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
 'Experience developing and using microservices']

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Nike's AIML Platform

Dear Hiring Manager,

I came across the job description for a Software Engineer II, AIML Platform at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you design and implement core components of your data science services and platform. With our expertise in computer science fundamentals, programming languages like Java, Python, and Golang, and experience with Infrastructure as Code / DevOps practices and technologies like Terraform, we're confident in our ability to deliver scalable, performant, and reliable solutions.

Our team has a strong background in developing and using microservices, and we're well-versed in Cloud services such as AWS Cloud EC2, S3, DynamoDB, and Azure Cloud. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in machine learning an